In [ ]:
import requests
import pandas as pd
import time
import math
from pathlib import Path
from pandas.io.parsers import read_csv
from google.colab import drive
import json

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 pip install pytrends

In [ ]:
# Función para extraer en un dataframe los eventos clasificados como musicales en TicketMaster
def datos_ticketmaster():
  pag = 0
  url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
  r = requests.get(url)

  if r.status_code == 200:
    respuesta = r.json()
    total_pages = respuesta['page']['totalPages']
    print(total_pages)
    df_datos = pd.DataFrame(respuesta['_embedded']['events'])
    pag += 1
    while(pag < total_pages):
      url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
      r = requests.get(url)
      res = r.json()
      df_datos_mas = pd.DataFrame(res['_embedded']['events'])
      df_datos = pd.concat([df_datos, df_datos_mas], ignore_index=True)
      pag += 1
  else:
    print('Error: ' + r.status_code)
    print(r.text)

  print(len(df_datos))
  return df_datos


In [ ]:
df_datos.info()

NameError: name 'df_datos' is not defined

In [ ]:
print(df_datos['type'].unique())    # la columna type toma únicamente el valor 'event' -> eliminar
df_datos_menos_col = df_datos.drop(columns = ['test', 'url','images',
                                              'promoters', 'type', 'locale',
                                              '_links', 'seatmap'])
df_datos_menos_col

['event']


,name,id,sales,dates,classifications,promoter,priceRanges,_embedded
0,Servicio de Autobús - BRESH - Grand Opening -...,Z698xZ2qZaa_M,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
1,BRESH - Grand Opening Concert Music Festival,Z698xZ2qZaa_S,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
2,Manolo García - Concert Music Festival,Z698xZ2qZaaC4,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
3,Servicio de Autobús - Manolo García - Concert...,Z698xZ2qZaaC9,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
4,Myke Towers + DJs invitados - Concert Music Fe...,Z698xZ2qZaa89,{'public': {'startDateTime': '2023-12-07T15:00...,{'access': {'startDateTime': '2024-07-04T20:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
...,...,...,...,...,...,...,...,...
886,"The Music of El Señor de los Anillos,El Hobbit...",Z698xZ2qZaFWs,{'public': {'startDateTime': '2023-08-09T15:28...,{'access': {'startDateTime': '2025-03-02T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty..."
887,The Magical Music of Harry Potter,Z698xZ2qZaaoh,{'public': {'startDateTime': '2023-12-07T16:17...,{'access': {'startDateTime': '2025-03-06T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Palau de Congressos de T...
888,Barcelona Bar Showroom,Z698xZ2qZaar2,{'public': {'startDateTime': '2024-02-13T16:26...,"{'start': {'localDate': '2024-03-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6341', 'name': 'BN GRUP 2019 S.L.'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Hyatt Regency Barcelona ...
889,"ATRÉVETE, el musical",Z698xZ2qZaaNL,{'public': {'startDateTime': '2024-02-21T13:30...,{'access': {'startDateTime': '2024-04-27T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5785', 'name': 'Producciones Comediart...","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Casa de la Cultura de Mu...


In [ ]:
df_datos_menos_col.iloc[2]['classifications']

[{'primary': True,
  'segment': {'id': 'KZFzniwnSyZfZ7v7nn', 'name': 'Film'},
  'genre': {'id': 'KnvZfZ7vAkJ', 'name': 'Music'},
  'subGenre': {'id': 'KZazBEonSMnZfZ7vave', 'name': 'Music'},
  'family': False}]

In [ ]:
# Definir una función para extraer el valor startDateTime de cada fila
def obtener_start_datetime(fila):
    return fila['dates']['access']['startDateTime']

df_datos_menos_col.loc[0, 'startDateTime']= obtener_start_datetime(df_datos_menos_col.iloc[0])
df_datos_menos_col


NameError: name 'df_datos_menos_col' is not defined

In [ ]:

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAME = "Taylor Swift"
DATE = "04-02-2024"  # Ajusta según sea necesario

# Construye la URL para solicitar datos de Last.fm
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={ARTIST_NAME}&api_key={LASTFM_API_KEY}&format=json"

# Realiza la solicitud a la API de Last.fm
response = requests.get(url)
data = response.json()

# Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
if "artist" in data and "stats" in data["artist"]:
    listeners = data["artist"]["stats"]["listeners"]
    print(f"Número de oyentes de {ARTIST_NAME} en {DATE}: {listeners}")
else:
    print(f"No se encontró información para {ARTIST_NAME} en la fecha {DATE}.")


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Nombre del artista que deseas buscar
    artist_name = 'Taylor Swift'

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {
        'q': artist_name,
        'type': 'artist',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Obtener el ID del primer resultado (artista)
    if 'artists' in search_data and 'items' in search_data['artists'] and len(search_data['artists']['items']) > 0:
        artist_id = search_data['artists']['items'][0]['id']
        print(f'El ID del artista {artist_name} es: {artist_id}')

    else:
        print(f'No se encontró al artista {artist_name}')

else:
    print('Error al obtener el token de acceso')
    print(token_data)


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # ID del artista que deseas consultar (puedes obtenerlo de la API de búsqueda)
    artist_id = '06HL4z0CvFAxyc27GXpf02'

    # URL de la API de Spotify para obtener información del artista
    artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud para obtener información del artista
    artist_response = requests.get(artist_url, headers=headers)
    artist_data = artist_response.json()

    # Verificar si se obtuvo correctamente la información del artista
    if 'followers' in artist_data:
        followers_count = artist_data['followers']['total']
        print(f'El artista tiene {followers_count} oyentes mensuales.')

    else:
        print('Error al obtener información del artista')
        print(artist_data)

else:
    print('Error al obtener el token de acceso')
    print(token_data)


In [ ]:
path = Path.cwd()   # 21/2/24 -> 39 paginas - 772 filas

print(path)
p = Path(path,"datos_21_02.csv")
df_datos.to_csv(p,index=False)
print(p)

In [ ]:
df_datos_menos_col

In [ ]:
def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        return fila['priceRanges'][0]['min']
    else:
        return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        return fila['priceRanges'][0]['max']
    else:
        return None

def venue_name(fila):
    return fila['_embedded']['venues'][0]['name']

def obtener_start_datetime(fila):
        return fila['sales']['public'].get('startDateTime')

def obtener_end_datetime(fila):
    return fila['sales']['public'].get('endDateTime')


# Aplicar las funciones a cada fila de la columna 'prices' y asignar los resultados a nuevas columnas 'min_price' y 'max_price'
df_datos_menos_col['min_price'] = df_datos_menos_col.apply(obtener_min_price, axis=1)
df_datos_menos_col['max_price'] = df_datos_menos_col.apply(obtener_max_price, axis=1)
df_datos_menos_col.iloc[0]['_embedded']

df_datos_menos_col['StartTimeSale'] = df_datos_menos_col.apply(obtener_start_datetime, axis=1)
df_datos_menos_col['EndTimeSale'] = df_datos_menos_col.apply(obtener_end_datetime, axis=1)

df_datos_menos_col['VenueName'] = df_datos_menos_col.apply(venue_name, axis=1)

# #print(df_datos_menos_col.iloc[1]['priceRanges'])
#df_datos_menos_col.iloc[0]['sales']
df_datos_menos_col.iloc[2]['_embedded']
#df_datos_menos_col

NameError: name 'df_datos_menos_col' is not defined

In [ ]:
def venue_name(fila):
    return fila['_embedded']['venues'][0]['name']

def venue_city(fila):
  return fila['_embedded']['venues'][0]['city']['name']

def venue_state(fila):
  return fila['_embedded']['venues'][0]['state']['name']

def venue_country(fila):
  return fila['_embedded']['venues'][0]['country']['name']

df_datos_menos_col['VenueName'] = df_datos_menos_col.apply(venue_name, axis=1)
df_datos_menos_col['VenueCity'] = df_datos_menos_col.apply(venue_city, axis=1)
df_datos_menos_col['VenueState'] = df_datos_menos_col.apply(venue_state, axis=1)
df_datos_menos_col['VenueCountry'] = df_datos_menos_col.apply(venue_country, axis=1)

df_datos_menos_col.iloc[0]['_embedded']

NameError: name 'df_datos_menos_col' is not defined

In [ ]:



def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['min']
    return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['max']
    return None

taylor = df_datos[df_datos['name'].str.contains('Taylor Swift')]
taylor = taylor.drop(columns = ['test', 'url','images',
                                              'promoters', 'type', 'locale',
                                              '_links', 'seatmap'])
# Aplicar las funciones a cada fila de la columna 'prices' y asignar los resultados a nuevas columnas 'min_price' y 'max_price'
taylor['min_price'] = taylor.apply(obtener_min_price, axis=1)
taylor['max_price'] = taylor.apply(obtener_max_price, axis=1)

taylor['StartTimeSale'] = taylor.apply(obtener_start_datetime, axis=1)
taylor['EndTimeSale'] = taylor.apply(obtener_end_datetime, axis=1)

taylor['VenueName'] = taylor.apply(venue_name, axis=1)
taylor['VenueCity'] = taylor.apply(venue_city, axis=1)
taylor['VenueState'] = taylor.apply(venue_state, axis=1)
taylor['VenueCountry'] = taylor.apply(venue_country, axis=1)

print(taylor.iloc[0]['priceRanges'])
print(taylor.iloc[1]['priceRanges'])
taylor

NameError: name 'df_datos' is not defined

In [ ]:
taylor.iloc[0]['_embedded']['attractions'][0]['classifications']

In [ ]:

def genre_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['genre']['name']

def subgenre_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['subGenre']['name']

def type_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['type']['name']

def subtype_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['subType']['name']

taylor.iloc[0]['_embedded']['attractions'][0]['classifications'][0]['genre']['name']

taylor['genre'] = taylor.apply(genre_event, axis = 1)
taylor['subGenre'] = taylor.apply(subgenre_event, axis = 1)
taylor['type'] = taylor.apply(type_event, axis = 1)
taylor['subType'] = taylor.apply(subtype_event, axis = 1)

taylor

NameError: name 'taylor' is not defined

In [ ]:
taylor.iloc[0]['classifications']

JUNTAR TODOS LOS DATOS Y FILTRAR


In [ ]:
d1 = read_csv('/content/drive/MyDrive/PD1/d_17_02.csv', header=0)
d2 = read_csv('/content/drive/MyDrive/PD1/d_1_03.csv', header=0)
d3 = read_csv('/content/drive/MyDrive/PD1/d_21_02.csv', header=0)
d4 = read_csv('/content/drive/MyDrive/PD1/d_26_02.csv', header=0)
d5 = read_csv('/content/drive/MyDrive/PD1/d_28_02.csv', header=0)
d6 = read_csv('/content/drive/MyDrive/PD1/datos_5_03.csv', header=0)

In [ ]:
# Juntar todos los dataframes en uno solo
df_total = pd.concat([d1, d2, d3, d4, d5, d6], ignore_index=True)
print(len(df_total))

# Contar las filas que están duplicadas
num_duplicados = df_total.duplicated().sum()
print("Filas duplicadas", num_duplicados)

# Eliminar los duplicados del dataframe (inplace=True para que los elimine sobre el propio dataframe)
df_total.drop_duplicates(inplace=True)
print('Num filas sin duplicados', len(df_total))

df_total = df_total.drop(columns = ['id', 'test', 'url','images', 'promoters',
                                    'type', 'locale', '_links', 'seatmap'])

5129
Filas duplicadas 2637
Num filas sin duplicados 2492


In [ ]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2492 entries, 0 to 5128
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             2492 non-null   object
 1   sales            2492 non-null   object
 2   dates            2492 non-null   object
 3   classifications  2492 non-null   object
 4   promoter         2489 non-null   object
 5   priceRanges      2483 non-null   object
 6   _embedded        2492 non-null   object
dtypes: object(7)
memory usage: 155.8+ KB


In [ ]:
# lista de strings a filtrar
strings_filtrar = ['film', 'servicio de autobús', 'festival', 'vip', 'meet', 'meet & greet', 'ticket', 'package', 'musical']

# Filtrar
for string in strings_filtrar:
    df_total = df_total[~df_total['name'].str.lower().str.contains(string)]

print(len(df_total))

1911


In [ ]:
df_total.sample(5)

,name,sales,dates,classifications,promoter,priceRanges,_embedded
1671,Homenaje a John Williams,{'public': {'startDateTime': '2024-01-30T17:21...,{'access': {'startDateTime': '2024-12-07T18:45...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5703', 'name': 'Overlook Events SAS'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Forum CCIB', '..."
4212,Black Coffee,{'public': {'startDateTime': '2024-02-23T16:30...,"{'start': {'localDate': '2024-10-05', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Hï Ibiza', 'type': 'venu..."
3171,David Guetta & Morten present FUTURE RAVE,{'public': {'startDateTime': '2024-02-26T11:00...,"{'start': {'localDate': '2024-09-27', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Hï Ibiza', 'type': 'venu..."
2438,Armin van Buuren at Ushuaïa Ibiza,{'public': {'startDateTime': '2024-02-14T17:30...,"{'start': {'localDate': '2024-09-29', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Ushuaïa Ibiza', 'type': ..."
4439,Marlon,{'public': {'startDateTime': '2023-11-07T11:00...,{'access': {'startDateTime': '2024-05-17T19:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '2771', 'name': 'Promociones Musicales ...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Razzmatazz 1', 'typ..."


Una vez eliminados algunos eventos por el nombre subdividimos las columnas que tenemos. Sin embargo, vemos que los valores de las filas usan comillas simples (') en vez de dobles (") y los valores booleanos en mayúsculas (False, True) en vez de minúsculas (false, true). Será necesario usar una función que corrija estos fallos y así podamos usar la función json.loads que nos permite convertir cadenas a diccionarios.





In [ ]:
def corregir_json(json_str):
    # Convertir a cadena de texto si no es una cadena
    if isinstance(json_str, str):
        # Reemplazar comillas simples por comillas dobles
        corregida = json_str.replace("'", '"')
        # Reemplazar "True" por "true" y "False" por "false"
        corregida = corregida.replace("True", "true")
        corregida = corregida.replace("False", "false")
        return json.loads(corregida)
    else:
        return json_str


def corregir_columnas(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].apply(corregir_json)
    return df

# Lista de columnas a corregir
columnas_a_corregir = ['sales', 'dates', 'classifications', 'promoter', 'priceRanges']

# Aplicar la función de corrección a las columnas especificadas
df_1 = corregir_columnas(df_total, columnas_a_corregir)

Usamos diferentes funciones para subdivdir las columnas.


In [ ]:
# Para obtener la start_datetime
def obtener_start_datetime(fila):
    if 'access' in fila['dates'] and 'startDateTime' in fila['dates']['access']:
        return fila['dates']['access']['startDateTime']
    else:
        return None  # O un valor predeterminado que desees asignar cuando la clave no está presente

def obtener_end_datetime(fila):
    return fila['sales']['public'].get('endDateTime')

def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['min']
    return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['max']
    return None

def obtener_promotor(fila):
    if 'promoter' in fila and isinstance(fila['promoter'], dict):
        return fila['promoter'].get('name')
    else:
        return None

def obtener_genero(fila):
    if isinstance(fila['classifications'], list) and fila['classifications']:
        # Verifica si classifications es una lista no vacía
        for item in fila['classifications']:
            if 'genre' in item and isinstance(item['genre'], dict):
                return item['genre'].get('name')
    return None

def obtener_subgenero(fila):
    if isinstance(fila['classifications'], list) and fila['classifications']:
        for item in fila['classifications']:
            if 'subGenre' in item and isinstance(item['subGenre'], dict):
                return item['subGenre'].get('name')
    return None


In [ ]:
df_1['startDateTime'] = df_1.apply(obtener_start_datetime, axis=1)
df_1['end_datetime'] = df_1.apply(obtener_end_datetime, axis=1)
df_1['min_price'] = df_1.apply(obtener_min_price, axis=1)
df_1['max_price'] = df_1.apply(obtener_max_price, axis=1)
df_1['genero'] = df_1.apply(obtener_genero, axis=1)
df_1['subgnero'] = df_1.apply(obtener_subgenero, axis=1)
df_1['promoter'] = df_1.apply(obtener_promotor, axis=1)

In [ ]:
df_1 = df_1.reset_index(drop=True)
df_1.sample(5)

,name,sales,dates,classifications,promoter,priceRanges,_embedded,startDateTime,end_datetime,min_price,max_price,genero,subgnero
924,Defected,{'public': {'startDateTime': '2024-02-14T17:00...,"{'start': {'localDate': '2024-07-30', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","Only Tickets, S.L.,","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Ushuaïa Ibiza', 'type': ...",None,2024-07-30T06:00:00Z,40.0,60.0,Dance/Electronic,Dance/Electronic
30,Fran Perea,{'public': {'startDateTime': '2023-09-26T10:00...,"{'start': {'localDate': '2024-02-23', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","Promociones Musicales Get In, S.L.","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'La Riviera', 'type': 've...",None,2024-02-23T20:00:00Z,25.0,38.0,Rock,Pop
604,Marc Seguí - AAAAAA Tour,{'public': {'startDateTime': '2023-10-31T15:00...,{'access': {'startDateTime': '2024-05-03T21:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",Taste the Floor S.L.,"[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Las Armas', 'type':...",2024-05-03T21:00:00Z,2024-05-03T20:00:00Z,20.0,20.0,Rock,Pop
59,Rulo y la Contrabanda - Tour 5,{'public': {'startDateTime': '2023-10-19T10:00...,{'access': {'startDateTime': '2024-03-15T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","Promociones Musicales Get In, S.L.","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Santana 27', 'type'...",2024-03-15T20:00:00Z,2024-03-15T20:00:00Z,30.0,30.0,Rock,Pop
1096,Kamelot + Ad Infinitum + Blackbriar + Frozen C...,{'public': {'startDateTime': '2024-02-16T10:00...,{'access': {'startDateTime': '2024-10-22T18:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",Sergio Solís Gálvez,"[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Sala Santana 27', 'type'...",2024-10-22T18:00:00Z,2024-10-22T16:30:00Z,32.0,32.0,Rock,Hard Rock


In [ ]:
df_1 = df_1.drop(columns=['sales','dates','classifications','priceRanges'])

In [ ]:
df_1.sample(5)

,name,promoter,_embedded,startDateTime,end_datetime,min_price,max_price,genero,subgnero
1751,Orphaned Land,Rubén Rosas Sánchez,"{'venues': [{'name': 'Sala Razzmatazz 2', 'typ...",2024-12-08T18:00:00Z,2024-12-08T17:00:00Z,27.00,27.00,Rock,Hard Rock
882,Camilo,"Tickex Experience, S.L.","{'venues': [{'name': 'Auditorio Starlite', 'ty...",2024-07-08T20:00:00Z,2024-07-06T21:59:00Z,76.79,177.68,Undefined,Undefined
295,Patriarchy,Albert Pavía Izquierdo,"{'venues': [{'name': 'Sala Specka', 'type': 'v...",2024-03-22T20:00:00Z,2024-03-22T22:59:00Z,19.00,19.00,Alternative,Alternative Rock
1333,Marc Anthony,"Sube el Volumen, A.I.E",{'venues': [{'name': 'Oasisss Marbella. Finca ...,2024-06-21T20:00:00Z,2024-06-21T20:00:00Z,59.00,221.00,Rock,Pop
400,Fiesta de Presentación del Vida 10º aniversario,"Sitback Produccions, S.L.","{'venues': [{'name': ""Masia d'En Cabanyes"", 't...",None,2024-05-11T10:00:00Z,23.00,23.00,Undefined,Undefined


Exploramos las nuevas columnas que hemos creado.


In [ ]:
generos = df_1['genero'].value_counts()
generos

Dance/Electronic     712
Rock                 692
Other                198
Undefined            107
Alternative           93
Hip-Hop/Rap           34
Music                 25
World                 25
Classical             11
Theatre                6
Blues                  4
Pop                    3
Fairs & Festivals      1
Name: genero, dtype: int64

In [ ]:
subgeneros = df_1['subgnero'].value_counts()
subgeneros

Dance/Electronic     712
Pop                  646
Other                198
Undefined            107
Alternative Rock      93
Hard Rock             46
Hip-Hop/Rap           34
Music                 25
Flamenco              16
Classical/Vocal       11
World                  9
Drama                  6
Jazz Blues             4
African Pop            3
Fairs & Festivals      1
Name: subgnero, dtype: int64

Es probable que no encontremos información de los oyenetes de los géneros: Classical, Theatre, Fairs & Festivals. Esta será una de nuestras variables principales por lo que al no tenerla no podremos usar estos datos como prueba.


In [ ]:
generos = ['Classical', 'Theatre', 'Fairs & Festivals']
df_1 = df_1[~df_1['genero'].isin(generos)]
len(df_1)

1893

Ahora queda subdividir la columna _embedded que se ha leido del csv como string. Necesitamos tranformarla en el diccionario que era para poder obtener más variables. Esta es una muestra de los valores que tiene:


In [ ]:
df_1.iloc[0]['_embedded']

"{'venues': [{'name': 'Auditorio Starlite', 'type': 'venue', 'id': 'Z598xZ2qZkAek', 'test': False, 'url': 'https://www.ticketmaster.es/venue/auditorio-starlite-marbella-entradas/starlitmar/114', 'locale': 'en-us', 'postalCode': '29602', 'timezone': 'Europe/Madrid', 'city': {'name': 'Marbella'}, 'state': {'name': 'Málaga'}, 'country': {'name': 'Spain', 'countryCode': 'ES'}, 'address': {'line1': 'Calle Albinoni, s/n. Cantera de Nagüeles'}, 'location': {'longitude': '-4.94999', 'latitude': '36.5266'}, 'upcomingEvents': {'mfx-es': 33, '_total': 33, '_filtered': 0}, '_links': {'self': {'href': '/discovery/v2/venues/Z598xZ2qZkAek?locale=en-us'}}}], 'attractions': [{'name': 'Jamie Cullum', 'type': 'attraction', 'id': 'K8vZ9175xH0', 'test': False, 'url': 'https://www.ticketmaster.co.uk/jamie-cullum-tickets/artist/907815', 'locale': 'en-us', 'externalLinks': {'youtube': [{'url': 'https://www.youtube.com/channel/UCOrI5RFAhBgAGcOzCUVbeNQ'}], 'twitter': [{'url': 'https://twitter.com/jamiecullum'}]

Al igual que antes reemplazamos las comillas simples por comillas dobles y los valores True, False a true, false. Hacemos estas modificaciones y se lo devolvemos al dataframe con el que estamos trabajando (df_1)


In [ ]:
em = df_1['_embedded']
col = em.str.replace("'", '"')
col = col.str.replace("True", 'true')
col = col.str.replace("False", 'false')
df_1['_embedded'] = col

Ahora arreglamos el los valores de la columna _embedded en una nueva columna llamada columna_json. En caso de que sea incapaz de cargar la columna, devolverá None


In [ ]:
def load_json(json_str):
    try:
        json_object = json.loads(json_str)
        return json_object
    except json.JSONDecodeError as e:
        return None

df_1['columna_json'] = df_1['_embedded'].apply(load_json)

print('Valores None en la columna',df_1['columna_json'].isna().sum())

Valores None en la columna 699


Como usaremos las variables de la columna_json creamos un nuevo dataframe (df_2) con las filas que no tiene none en la columna_json.

In [ ]:
df_2 = df_1.dropna(subset=['columna_json'])
len(df_2)

1194

Ahora dividimos la columna_json en nuevas variables.

In [ ]:
def genre_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['genre']['name']
    else:
        return None

def subgenre_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['subGenre']['name']
    else:
        return None

def type_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['type']['name']
    else:
        return None

def subtype_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['subType']['name']
    else:
        return None

def name(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['name']
    else:
        return None

def venue_name(fila):
    return fila['columna_json']['venues'][0]['name']

def venue_city(fila):
  return fila['columna_json']['venues'][0]['city']['name']

def venue_state(fila):
  return fila['columna_json']['venues'][0]['state']['name']

def venue_country(fila):
  return fila['columna_json']['venues'][0]['country']['name']



In [ ]:
df_2.loc[:, 'nameArtist'] = df_2.apply(name, axis=1).copy()
df_2.loc[:, 'genre'] = df_2.apply(genre_event, axis=1).copy()
df_2.loc[:, 'subgenre'] = df_2.apply(subgenre_event, axis=1).copy()
df_2.loc[:, 'type'] = df_2.apply(type_event, axis=1).copy()
df_2.loc[:, 'subtype'] = df_2.apply(subtype_event, axis=1).copy()
df_2['VenueName'] = df_2.apply(venue_name, axis=1)
df_2['VenueCity'] = df_2.apply(venue_city, axis=1)
df_2['VenueState'] = df_2.apply(venue_state, axis=1)
df_2['VenueCountry'] = df_2.apply(venue_country, axis=1)

<ipython-input-99-796e21fc6b3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'nameArtist'] = df_2.apply(name, axis=1).copy()
<ipython-input-99-796e21fc6b3d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'genre'] = df_2.apply(genre_event, axis=1).copy()
<ipython-input-99-796e21fc6b3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [ ]:
df_2 = df_2.drop(columns=['_embedded','columna_json','VenueCountry',])

In [ ]:
df_2

,name,promoter,startDateTime,end_datetime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga
1906,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid
1907,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid
1908,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Lista de nombres de artistas
    artist_names = [i for i in df_2['nameArtist'].unique()]

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'

    # Configurar el diccionario para almacenar los IDs de los artistas
    artist_ids = {}

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista para cada nombre en la lista
    for artist_name in artist_names:
        search_params = {
            'q': artist_name,
            'type': 'artist',
        }

        # Hacer la solicitud de búsqueda del artista
        search_response = requests.get(search_url, params=search_params, headers=headers)
        search_data = search_response.json()

        # Verificar si se encontraron resultados para el artista en 2022
        if 'artists' in search_data and 'items' in search_data['artists']:
            for artist in search_data['artists']['items']:
                if 'name' in artist and artist['name'] == artist_name:
                    # Agregar la entrada al diccionario
                    artist_ids[artist_name] = artist['id']
                    break
            else:
                print(f"No se encontraron resultados para {artist_name} en 2022")
        else:
            print(f'No se encontró al artista {artist_name}')

    # Imprimir el diccionario resultante

else:
    print('Error al obtener el token de acceso')
    print(token_data)


No se encontraron resultados para UB40 Featuring Ali Campbell in memory of Astro en 2022
No se encontraron resultados para Ballet Flamenco Sara Baras en 2022
No se encontraron resultados para Australian Pink Floyd Show en 2022
No se encontraron resultados para Rulo y la Contrabanda en 2022
No se encontraron resultados para Dystinct en 2022
No se encontraron resultados para Vida Records & Friends en 2022
No se encontraron resultados para Álvaro de Luna en 2022
No se encontraron resultados para Pablo Rouss en 2022
No se encontró al artista None
No se encontraron resultados para Primavera Pop en 2022
No se encontraron resultados para Tan Biónica en 2022
No se encontraron resultados para MANÁ en 2022
No se encontraron resultados para gREAT sTRAITS en 2022
No se encontraron resultados para Rozalen en 2022
No se encontraron resultados para Stingers en 2022
No se encontraron resultados para Mago De Oz en 2022
No se encontraron resultados para Tributo a Queen en 2022
No se encontraron resultad

In [ ]:
import pandas as pd
df_ids= pd.DataFrame([[key, artist_ids[key]] for key in artist_ids.keys()], columns=['Name', 'Ids'])
df_ids

,Name,Ids
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD
2,Aitana,7eLcDZDYHXZCebtQmVFL25
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP
4,Take That,1XgFuvRd7r5g0h844A5ZUQ
...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164
154,Simple Minds,6hN9F0iuULZYWXppob22Aj
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2
156,Front 242,2tyMOS8xKREgpEwHnLc6EX


In [ ]:
import requests
import pandas as pd

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # DataFrame con la columna de IDs de los artistas
    df_artist_ids = pd.DataFrame({'Ids': df_ids['Ids']})

    # URL base de la API de Spotify para obtener información de varios artistas
    artists_url = 'https://api.spotify.com/v1/artists'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Lista para almacenar los números de oyentes de cada artista
    followers_counts = []

    # Hacer la solicitud para obtener información de cada artista
    for artist_id in df_artist_ids['Ids']:
        artist_url = f'{artists_url}/{artist_id}'
        artist_response = requests.get(artist_url, headers=headers)
        artist_data = artist_response.json()

        # Verificar si se obtuvo correctamente la información del artista
        if 'followers' in artist_data:
            followers_count = artist_data['followers']['total']
            followers_counts.append(followers_count)
        else:
            followers_counts.append(None)  # Si no se encuentra el número de oyentes, agregar None

    # Agregar la columna de números de oyentes al DataFrame
    df_artist_ids['followers_count'] = followers_counts

    # Mostrar el DataFrame resultante

else:
    print('Error al obtener el token de acceso')
    print(token_data)

df_artist_ids

,Ids,followers_count
0,3XxxEq6BREC57nCWXbQZ7o,514905
1,4TKTii6gnOnUXQHyuo9JaD,1119957
2,7eLcDZDYHXZCebtQmVFL25,2898099
3,0XwVARXT135rw8lyw1EeWP,14130881
4,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...
153,4asJQrazucWu3Zy5R7B164,3549
154,6hN9F0iuULZYWXppob22Aj,1935158
155,1c8m6rFeA6QIhImbIaZ7k2,106030
156,2tyMOS8xKREgpEwHnLc6EX,139712


In [ ]:
df_ids['Oyentes Mensuales']=df_artist_ids['followers_count']
df_ids

,Name,Ids,Oyentes Mensuales
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o,514905
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD,1119957
2,Aitana,7eLcDZDYHXZCebtQmVFL25,2898099
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP,14130881
4,Take That,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164,3549
154,Simple Minds,6hN9F0iuULZYWXppob22Aj,1935158
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2,106030
156,Front 242,2tyMOS8xKREgpEwHnLc6EX,139712


In [ ]:
df_ids.rename(columns={'Name': 'nameArtist'}, inplace=True)
df_ids

,nameArtist,Ids,Oyentes Mensuales
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o,514905
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD,1119957
2,Aitana,7eLcDZDYHXZCebtQmVFL25,2898099
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP,14130881
4,Take That,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164,3549
154,Simple Minds,6hN9F0iuULZYWXppob22Aj,1935158
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2,106030
156,Front 242,2tyMOS8xKREgpEwHnLc6EX,139712


In [ ]:
df3 = df_2.merge( df_ids, on='nameArtist', how='left')

In [ ]:
df3

,name,promoter,startDateTime,end_datetime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState,Ids,Oyentes Mensuales
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,3XxxEq6BREC57nCWXbQZ7o,514905.0
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,4TKTii6gnOnUXQHyuo9JaD,1119957.0
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,7eLcDZDYHXZCebtQmVFL25,2898099.0
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga,0XwVARXT135rw8lyw1EeWP,14130881.0
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1XgFuvRd7r5g0h844A5ZUQ,1278678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga,60uh2KYYSCqAgJNxcU4DA0,4122165.0
1190,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid,0XwVARXT135rw8lyw1EeWP,14130881.0
1191,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid,2nTjd2lNo1GVEfXM3bCnsh,675923.0
1192,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona,2nTjd2lNo1GVEfXM3bCnsh,675923.0


In [ ]:
import requests

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAMES = [i for i in df_datos_menos_col['nameArtist'].dropna().unique()]
DATE = "03-03-2024"  # Ajusta según sea necesario

for artist_name in ARTIST_NAMES:
    # Construye la URL para solicitar datos de Last.fm para cada artista
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={LASTFM_API_KEY}&format=json"

    # Realiza la solicitud a la API de Last.fm
    response = requests.get(url)
    data = response.json()

    # Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
    if "artist" in data and "stats" in data["artist"]:
        listeners = data["artist"]["stats"]["listeners"]
        print(f"Número de oyentes de {artist_name}: {listeners}")
    else:
        print(f"No se encontró información para {artist_name}.")



Número de oyentes de Jamie Cullum: 1210924
Número de oyentes de Sheryl Crow: 1595785
Número de oyentes de Aitana: 151113
Número de oyentes de Christian Nodal: 161670
Número de oyentes de Take That: 1041706
Número de oyentes de Tom Jones: 1135326
Número de oyentes de Luis Miguel: 506120
Número de oyentes de Diana Krall: 1230182
Número de oyentes de Los Secretos: 112122
Número de oyentes de Antonio José: 30693
Número de oyentes de HAUSER: 27469
Número de oyentes de Miguel Poveda: 40423
Número de oyentes de Gordo: 38824
Número de oyentes de La Oreja de Van Gogh: 491497
Número de oyentes de Abraham Mateo: 112421
Número de oyentes de Siempre Así: 1962
Número de oyentes de Marc Seguí: 110118
Número de oyentes de Dead Poet Society: 152036
Número de oyentes de Dreamcatcher: 412195
Número de oyentes de Manolo Garcia: 12013
Número de oyentes de Hens: 19015
Número de oyentes de Café Quijano: 91382
Número de oyentes de Fran Perea: 21512
Número de oyentes de IAMDDB: 153607
Número de oyentes de The 

In [ ]:
import pandas as pd
import requests

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAMES = [i for i in df_datos_menos_col['nameArtist'].dropna().unique()]
DATE = "03-03-2024"  # Ajusta según sea necesario

# Listas para almacenar los datos
artist_list = []
listeners_list = []

for artist_name in ARTIST_NAMES:
    # Construye la URL para solicitar datos de Last.fm para cada artista
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={LASTFM_API_KEY}&format=json"

    # Realiza la solicitud a la API de Last.fm
    response = requests.get(url)
    data = response.json()

    # Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
    if "artist" in data and "stats" in data["artist"]:
        listeners = data["artist"]["stats"]["listeners"]
        artist_list.append(artist_name)
        listeners_list.append(listeners)
    else:
        print(f"No se encontró información para {artist_name}.")

# Crea un DataFrame a partir de las listas de datos recopilados
df = pd.DataFrame({"Artist": artist_list, "Listeners": listeners_list})

# Muestra el DataFrame
df


No se encontró información para Funzo & Baby Loud.
No se encontró información para UB40 Featuring Ali Campbell in memory of Astro.
No se encontró información para Ballet Flamenco Sara Baras.
No se encontró información para Vida Records & Friends.
No se encontró información para Pablo Rouss.
No se encontró información para Primavera Pop.
No se encontró información para gREAT sTRAITS.
No se encontró información para Juanjo Monserrat & Mansfeld.
No se encontró información para Paula Mattheus en Concierto.
No se encontró información para Kadec Santa Ana.
No se encontró información para NieR: Orchestra Concert.
No se encontró información para Premier - La gran quedada.
No se encontró información para SO-LA-NA.
No se encontró información para The Music of Hans Zimmer & More.
No se encontró información para The Magical Music of Harry Potter - Live in Concert.
No se encontró información para Eurovision on tour.
No se encontró información para HIVERNACLE.
No se encontró información para Barcelo

,Artist,Listeners
0,Jamie Cullum,1210924
1,Sheryl Crow,1595785
2,Aitana,151113
3,Christian Nodal,161670
4,Take That,1041706
...,...,...
170,El Langui,1272
171,Eva Ruíz,4
172,Flor Bertotti,41
173,Ballet Nacional De Espana,3


In [ ]:
df=df.rename(columns={'Artist':'nameArtist'})

In [ ]:
df4 = df3.merge(df, on='nameArtist', how='left')

In [ ]:
df4=df4.drop(columns='Ids')

In [ ]:
df4=df4.rename(columns={'Oyentes Mensuales':'Spotify_Mensuales','Listeners':'Last.fm_Oyentes'})
df4

,name,promoter,startDateTime,end_datetime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState,Spotify_Mensuales,Last.fm_Oyentes
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,514905.0,1210924
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1119957.0,1595785
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,2898099.0,151113
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga,14130881.0,161670
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1278678.0,1041706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga,4122165.0,356132
1190,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid,14130881.0,161670
1191,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid,675923.0,180047
1192,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona,675923.0,180047
